In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from ipywidgets import interact, FloatSlider, Label
from shapely.geometry import Point
from shapely.plotting import plot_polygon

# Exercise 3: **Altimeter**

*Simulate and explain the echo of an altimeter observing a ﬂat terrain by assuming a constant power-pattern across the beamwidth.*

## *How does the measurement-principle of an altimeter work?*

An altimeter is a nadir-looking MW transmitting satellite. It transmits a pulse of duration $\tau$ and towards the earth's surface and registers the received echo. If the altitude of the satellite $H$ and the pulse's propagation velocity $v$ are known, the time delay can be related to a distance. That is, earth's surface topography is scanned. 

Let's assume that the power transmitted is not a function of the angle $\theta$, meaning the power is constant over the whole beamwidth.
As the pulse transmitted at $t_0$ is an "annulus-shaped" wavefront, it reaches earth's surface after a time $\tilde{t}_1 = \dfrac{v}{2}\left(t-t_0\right)$. Due to the shape of the impinging pulse, until the time $\tilde{t}_2 = \dfrac{v}{2}\left(t-t_0+\tau\right)$ the illimination pattern is a disc. It's radius can be apporoximated as $r \approx \sqrt{Hv\left(t-t_0\right)}$. The illuminated area is thus $A = \pi r^2 = \pi Hv\left(t-t_0\right)$, which increases linearly with $t$. As the received power is deirectly porportional to the illuminated area, it expresses the same behavior.

For times $t > \tilde{t}_2$, the illumination pattern becomes an annulus, with inner radius $r \approx \sqrt{Hv\left(t-t_0-\tau\right)}$ and outer radius $R \approx \sqrt{Hv\left(t-t_0\right)}$. Thus, the illuminated area is given as $A \approx \pi H v \tau$, which is independent of time and only depends on the pulse duration and the satellite's altitude. Similarly, the receiverd power os constant.

As the sensor has an aperture limiting the received signal to certain incident angles, the received power eventually decrreases lienarly back to zero again. Simply because the signal is no longer detected, not because it would not exist.

## *What happens if the transmitted power across the beamwidth is not isotropic?*

In a real antenna, the transmitted signal power is a function of the angle $\theta$, dictated by the underlying antenna radiation pattern. Thus, as the transmitted signal is not a delta function but a beam of width $\beta = 2\theta$, the power will not be constant across the beamwidth. That means, that received signals echoing back from an icnreasingly larger angle $\theta$ will be of lower power, even if the surface is ideal. Thus, the real received power is less than would be expecetd.

## *What is the diﬀerence between a pulse-limited and a beam-limited altimeter?*

The radius of the illumination pattern can be obtained using simple trigonometric relations, revealing $r_{ill} = H \tan\theta$.

* **Pulse-limited** if $r_{ill} \gg \sqrt{Hv\tau}$: non-isotropic power distribution over beamwidth negligible, foorprint radius given as $r_p = \sqrt{Hv\tau}$
* **Beam-limited** if $r_{ill} \ll \sqrt{Hv\tau}$: non-isotropic power distribution over beamwidth **not** negligible, foorprint radius given as $r_p = H\tan\theta = H \tan\left(\dfrac{\beta}{2}\right)$

## *How does a rough-surface aﬀect the measured signal?*

A rough surface transforms the received signal into a function of topography. While this is exactly the purpose of an altimeter, the behavior of the received signal's power becomes more tricky. Depending on the altitude distribution in the ground, additional time delays are introduced to the signal. Overall, this results in a smoother curve for the received power.

In [11]:
H = 800e3                       #m
THETA = 5                       #deg
TAU = 5e-3                     #s
T0 = 0                          #s
SPEED_OF_LIGHT = 299792458    #m/s
FACTOR = 1000
SPEED_OF_LIGHT = SPEED_OF_LIGHT / FACTOR

In [12]:
def illumination_pattern(t, H, THETA, TAU, T0):
    x = H * np.tan(np.deg2rad(THETA))
    if (t - T0) <= TAU:
        r1 = np.sqrt(H * SPEED_OF_LIGHT * (t - T0)) * np.sqrt(1 + (SPEED_OF_LIGHT * (t - T0)) / (4 * H))
        r0 = 0
    elif (t - T0) > TAU:
        r0 = np.sqrt(H * SPEED_OF_LIGHT * (t - T0 - TAU))
        r1 = np.sqrt(H * SPEED_OF_LIGHT * (t - T0))
    else:
        r0, r1 = 0, 0
    if r1 > x:
        r1 = x
    if r0 > x:
        r0 = x
    return x, Point(0, 0).buffer(r1), Point(0, 0).buffer(r0)


In [18]:

time_slider = FloatSlider(value = T0, min = 0, max = T0 + 20 * TAU, step = 0.001, description = 't [s]:')
theta_slider = FloatSlider(value=THETA, min=0, max=15, step=0.1, description='Theta [deg]:')
tau_slider = FloatSlider(value=TAU, min=0, max=0.1, step=0.001, description='tau [s]:')

@interact(t = time_slider, theta=theta_slider, tau = tau_slider)
def update_plot(t, theta, tau):
    fig = plt.figure(figsize=(16, 6))


    fig.suptitle(f'Aperture limited (theta: {theta:.2f}deg) illumintaion pattern, beamwidth with isotropic power distribution\n Satellite altitude {(H/1000):.2f}km, signal propagation velocity c/{FACTOR}', fontsize = 12)
    ax1 = fig.add_subplot(121)
    ax1.set_aspect(1)
    
    max_rad, c1, c0 = illumination_pattern(t, H, theta, tau, T0)
    res = c1.difference(c0).area / (1e6)

    ax1.set_title(f'pulse duration {tau:.3f}s:\nIlluminated area: {res:.2f}km^2')
    plot_polygon(c1, ax=ax1, add_points=False, color='grey', alpha=0.5)
    plot_polygon(c0, ax=ax1, add_points=False, color='white', alpha=1)
    ax1.set_xlim(-1.1*max_rad, 1.1*max_rad)
    ax1.set_ylim(-1.1*max_rad, 1.1*max_rad)
    
    ax1.set_xlabel('Distance [m]')
    ax1.set_ylabel('Distance [m]')
    ax1.ticklabel_format(axis='both', style='sci')

    ax2 = fig.add_subplot(122)

    ttt = np.linspace(0, T0 + 20 * TAU, 1000)
    area = []
    for tt in ttt:
        max_rad, c1, c0 = illumination_pattern(tt, H, theta, tau, T0)
        res = c1.difference(c0).area / (1e6)
        area.append(res)

    ax2.plot(ttt, area, linewidth = 2, color = 'grey', label = 'Ideal case')

    max_rad, c1, c0 = illumination_pattern(t, H, theta, tau, T0)
    res = c1.difference(c0).area / (1e6)
    ax2.plot(t, res, linestyle = 'None', marker = 'o', markersize = 10, color = 'red', label = f't={t:.4f}s')

    ax2.set_ylabel('Received Power Equivalent')
    ax2.set_xlabel('Time [s]')
    plt.legend()


    plt.tight_layout()



interactive(children=(FloatSlider(value=0.0, description='t [s]:', max=0.1, step=0.001), FloatSlider(value=5.0…